In [102]:
#https://nlpforhackers.io/topic-modeling/
import nltk

#nltk.download('brown')
from nltk.corpus import brown
from elasticsearch import Elasticsearch
#https://www.elastic.co/blog/how-to-find-and-remove-duplicate-documents-in-elasticsearch
es = Elasticsearch(["localhost:9200"])
 
data = []

indexToSearch = 'intj'

def scroll_over_all_docs():
    resp = es.search(index=indexToSearch, scroll='1m',  body={"query": {"match_all": {}}})
    # Get the scroll ID
    #print(resp)
    sid = resp['_scroll_id']
    scroll_size = len(resp['hits']['hits'])
    # Before scroll, process current batch of hits
    print(scroll_size)
    #data.append(resp['hits']['hits']['_source']['section'])
    for hit1 in resp['hits']['hits']:
        if(hit1['_source'] and 'section' in hit1['_source']):
            data.append(hit1['_source']['section'])
    while scroll_size > 0:
        resp2 = es.scroll(scroll_id=sid, scroll='2m')
        # Process current batch of hits
        for hit in resp2['hits']['hits']:
            #print(hit['_source']['section'])
            if(hit['_source'] and 'section' in hit['_source']):
                data.append(hit['_source']['section'])
            else:
                print(hit['_source'])
                res = es.delete(
                index=hit['_index'], doc_type=hit['_type'], id=hit['_id'])
        # Update the scroll ID
        sid = resp2['_scroll_id']
        # Get the number of results that returned in the last scroll
        scroll_size = len(resp2['hits']['hits'])
 
# for fileid in brown.fileids():
#     document = ' '.join(brown.words(fileid))
#     data.append(document)

scroll_over_all_docs()
NO_DOCUMENTS = len(data)
print(NO_DOCUMENTS)
print(data[:5])

10
164
[' Introverted iNtuitive Thinking Judging Susan B. Anthony Lance Armstrong Arthur Ashe, tennis champion Augustus Caesar (Gaius Julius Caesar Octavianus) Jane Austen ( Pride and Prejudice ) Dan Aykroyd ( The Blues Brothers ) William J. Bennett, "drug czar" William F. Buckley, Jr. Raymond Burr ( Perry Mason, Ironsides ) Chevy Chase (Cornelius Crane) ( Fletch ) Katie Couric Phil Donahue Michael Dukakis, governor of Mass., 1988 U.S. Dem. pres. candidate Richard Gere ( Pretty Woman ) Rudy Giuliani, former New York City mayor Greg Gumbel, television sportscaster Hannibal, Carthaginian military leader Emily Bronte, author of Wuthering Heights Angela Lansbury ( Murder, She Wrote ) Orel Leonard Hershiser, IV Peter Jennings Charles Everett Koop Ivan Lendl C. S. Lewis ( The Chronicles of Narnia ) Joan Lunden Edwin Moses, U.S. olympian (hurdles) Martina Navratilova Michelle Obama General Colin Powell, former US Secretary of State Charles Rangel, US Representative, D-N.Y. Pernell Roberts ( B

In [103]:
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 2
STOPWORDS = stopwords.words('english')
 
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in data:
    tokenized_data.append(clean_text(text))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

<>:11: DeprecationWarning: invalid escape sequence \-
<>:11: DeprecationWarning: invalid escape sequence \-
<>:11: DeprecationWarning: invalid escape sequence \-
<ipython-input-103-12f11718d36f>:11: DeprecationWarning: invalid escape sequence \-
  cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
[(42, 1), (69, 1), (77, 2), (89, 1), (105, 1), (115, 1), (116, 3), (133, 1), (148, 1), (248, 2), (486, 1), (715, 1), (716, 1), (717, 1), (718, 1), (719, 1), (720, 1), (721, 1), (722, 1), (723, 1), (724, 1), (725, 1), (726, 1), (727, 1), (728, 1), (729, 1), (730, 2), (731, 1), (732, 1), (733, 1), (734, 1), (735, 1), (736, 2), (737, 1), (738, 2), (739, 1), (740, 1), (741, 1), (742, 1), (743, 2)]


In [104]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.014*"intj" + 0.013*"intjs" + 0.011*"ago" + 0.011*"type" + 0.009*"reply" + 0.009*"says" + 0.008*"personality" + 0.007*"love" + 0.006*"verified" + 0.006*"people"
Topic #1: 0.015*"intjs" + 0.013*"intj" + 0.011*"type" + 0.010*"personality" + 0.008*"verified" + 0.008*"reply" + 0.008*"people" + 0.007*"says" + 0.007*"ago" + 0.006*"types"
LSI Model:
Topic #0: 0.343*"ago" + 0.330*"reply" + 0.321*"says" + 0.296*"verified" + 0.210*"intj" + 0.180*"weeks" + 0.175*"love" + 0.156*"months" + 0.150*"people" + 0.148*"god"
Topic #1: 0.336*"type" + 0.308*"intjs" + 0.266*"intj" + 0.221*"personality" + -0.187*"ago" + -0.177*"reply" + -0.174*"says" + 0.169*"people" + 0.157*"thinking" + -0.157*"verified"


In [105]:
text = "ENTJs are very strong"
bow = dictionary.doc2bow(clean_text(text))
 
print(lsi_model[bow])
# [(0, 0.091615426138426506), (1, -0.0085557463300508351), (2, 0.016744863677828108), (3, 0.040508186718598529), (4, 0.014201267714185898), (5, -0.012208538275305329), (6, 0.031254053085582149), (7, 0.017529584659403553), (8, 0.056957633371540077),(9, 0.025989149894888153)
 
print(lda_model[bow])
# [(0, 0.020005183), (1, 0.020005869), (2, 0.02000626), (3, 0.020005472), (4, 0.020009108), (5, 0.020005926), (6, 0.81994385), (7, 0.020006068), (8, 0.020006327), (9, 0.020005994)]
 

[]
[(0, 0.5), (1, 0.5)]


In [106]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
# [(104, 0.87591344), (178, 0.86124849), (31, 0.8604598), (77, 0.84932965), (85, 0.84843522), (135, 0.84421808), (215, 0.84184396), (353, 0.84038532), (254, 0.83498049), (13, 0.82832891)]
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(data[document_id][:1000])

[(2, 0.99999994), (4, 0.99999994), (6, 0.99999994), (7, 0.99999994), (10, 0.99999994), (12, 0.99999994), (14, 0.99999994), (16, 0.99999994), (18, 0.99999994), (36, 0.99999994)]
 


In [107]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 2
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])

<>:8: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \-
<ipython-input-107-890a0a1cc74a>:8: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
(164, 2)
(164, 2)
(164, 2)
[0.16488413 0.83511587]
[0.01116903 0.68937973]
[3.60904653 6.91348935]


In [69]:

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)
 

LDA Model:
Topic 0:
[('type', 1787.8096172978715), ('personality', 1764.1713830571357), ('reply', 1561.4673366764105), ('style', 771.4826217043075), ('flirting', 764.6551463972368), ('myers-briggs', 740.3965359632107), ('infj', 664.82764743737), ('based', 646.0980286767805), ('comment', 329.5058955619542), ('psychology', 323.6265284988812)]
Topic 1:
[('reply', 561.0104127517188), ('ago', 475.2114557373172), ('says', 457.1031217730417), ('verified', 377.92032968672345), ('months', 270.99342993549334), ('like', 225.89854676036362), ('people', 212.7106016969141), ('just', 199.8959811613315), ('weeks', 173.97878485258445), ('don', 173.5410526722336)]
NMF Model:
Topic 0:
[('ago', 7.904444763010319), ('says', 7.716918391246289), ('reply', 7.591452105659959), ('verified', 6.52954121771586), ('months', 4.727749587299044), ('weeks', 2.6823818512487247), ('like', 2.6725741055096606), ('people', 2.330503845953068), ('years', 2.2389995146361352), ('just', 2.112658529166702)]
Topic 1:
[('type', 6.7

In [112]:

text = "The entj is very strong"
x = nmf_model.transform(vectorizer.transform([text]))[0]
print(x)

[0.         0.00282215]


In [113]:

from sklearn.metrics.pairwise import euclidean_distances
 
def most_similar(x, Z, top_n=5):
    dists = euclidean_distances(x.reshape(1, -1), Z)
    pairs = enumerate(dists[0])
    most_similar = sorted(pairs, key=lambda item: item[1])[:top_n]
    return most_similar
 
similarities = most_similar(x, nmf_Z)
document_id, similarity = similarities[0]
print(data[document_id][:1000])

In [114]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [116]:

svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)
 

In [61]:
print(data_vectorized)

(0, 744)	12
  (0, 578)	2
  (0, 797)	2
  (0, 1284)	2
  (0, 1475)	42
  (0, 1187)	51
  (0, 1504)	9
  (0, 303)	6
  (0, 31)	51
  (0, 486)	8
  (0, 794)	19
  (0, 560)	6
  (0, 559)	6
  (0, 489)	2
  (0, 147)	1
  (0, 1527)	1
  (0, 8)	1
  (0, 1402)	5
  (0, 997)	11
  (0, 259)	1
  (0, 218)	2
  (0, 1003)	4
  (0, 1490)	1
  (0, 807)	2
  (0, 1469)	4
  :	:
  (355, 1067)	1
  (355, 609)	1
  (355, 39)	1
  (355, 1272)	1
  (355, 888)	1
  (355, 728)	1
  (355, 552)	1
  (356, 1475)	1
  (356, 1187)	1
  (356, 303)	1
  (356, 31)	1
  (356, 997)	1
  (356, 1149)	1
  (356, 1543)	1
  (356, 339)	1
  (356, 630)	1
  (356, 1449)	1
  (356, 1502)	2
  (356, 243)	1
  (356, 316)	1
  (356, 670)	1
  (356, 1528)	1
  (356, 915)	2
  (356, 946)	1
  (356, 110)	1


In [81]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "Entjs are very strong"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1] 1.0
<>:8: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \-
<ipython-input-81-bafeaded7620>:8: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [82]:
print(lda_Z)

[[7.40041710e-02 1.17651190e-03 1.17687893e-03 1.17652485e-03
  2.12389951e-01 1.17656117e-03 7.05369883e-01 1.17650017e-03
  1.17651034e-03 1.17650697e-03]
 [1.63148969e-04 1.63137113e-04 1.63150969e-04 1.63136761e-04
  1.63158648e-04 1.63143682e-04 9.98531714e-01 1.63135658e-04
  1.63137206e-04 1.63136501e-04]
 [1.00000000e-01 1.00000000e-01 1.00000000e-01 1.00000000e-01
  1.00000000e-01 1.00000000e-01 1.00000000e-01 1.00000000e-01
  1.00000000e-01 1.00000000e-01]
 [1.63148969e-04 1.63137113e-04 1.63150969e-04 1.63136761e-04
  1.63158648e-04 1.63143682e-04 9.98531714e-01 1.63135658e-04
  1.63137206e-04 1.63136501e-04]
 [1.00000000e-01 1.00000000e-01 1.00000000e-01 1.00000000e-01
  1.00000000e-01 1.00000000e-01 1.00000000e-01 1.00000000e-01
  1.00000000e-01 1.00000000e-01]
 [1.63148969e-04 1.63137113e-04 1.63150969e-04 1.63136761e-04
  1.63158648e-04 1.63143682e-04 9.98531714e-01 1.63135658e-04
  1.63137206e-04 1.63136501e-04]
 [1.00000000e-01 1.00000000e-01 1.00000000e-01 1.00000000e

In [63]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
6      32.420952   8.115230       1        1  57.765108
3       0.648955  71.707100       2        1  29.059527
5      71.019684   9.971006       3        1  10.567480
7       3.785981  33.185135       4        1   1.582768
8     -42.666500  -9.222070       5        1   0.666974
4      -3.621689 -43.526516       6        1   0.134102
2      -3.632402  -4.171708       7        1   0.056011
0     -36.584400  35.804413       8        1   0.056010
9      44.391380  50.615383       9        1   0.056010
1      40.235809 -31.591665      10        1   0.056010, topic_info=             Term         Freq        Total Category  logprob  loglift
1441         type  1851.000000  1851.000000  Default  30.0000  30.0000
1006  personality  1839.000000  1839.000000  Default  29.0000  29.0000
1149        reply  2111.000000  2111.000000  Default  28.0000  28.0000
997        people   428.000000   428.000000  Default  27.0000  27.0000
31            ago   494.000000   494.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
791          life     0.019597   193.892081  Topic10  -7.2782  -1.7123
1381        tests     0.019545    37.294469  Topic10  -7.2809  -0.0665
1543        years     0.019452   217.859684  Topic10  -7.2857  -1.8363
594          hate     0.019409    33.051087  Topic10  -7.2879   0.0473
958        online     0.019406    21.906864  Topic10  -7.2880   0.4584

[703 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
6         2  0.340930   academic
6         3  0.681859   academic
9         1  0.380376  accepting
9         2  0.190188  accepting
9         4  0.380376  accepting
...     ...       ...        ...
1548      2  0.279160    younger
1548      3  0.093053    younger
1551      2  0.969511        yup
1552      1  0.658247       zone
1552      2  0.164562       zone

[1118 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 6, 8, 9, 5, 3, 1, 10, 2])

In [52]:
import string
import collections
 
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
 
 
def process_text(text, stem=True):
    """ Tokenize text and stem words removing punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
 
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens
 
 
def cluster_texts(texts, clusters=3):
    """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
    vectorizer = TfidfVectorizer(tokenizer=process_text,
                                 stop_words=stopwords.words('english'),
                                 max_df=0.5,
                                 min_df=0.1,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    km_model = KMeans(n_clusters=clusters)
    km_model.fit(tfidf_model)
 
    clustering = collections.defaultdict(list)
 
    for idx, label in enumerate(km_model.labels_):
        clustering[label].append(idx)
 
    return clustering
 
 
if __name__ == "__main__":
    articles = data
    clusters = cluster_texts(articles, 7)
    pprint(dict(clusters))
 

{0: [1,
     27,
     30,
     33,
     38,
     43,
     46,
     47,
     49,
     51,
     54,
     56,
     60,
     66,
     71,
     79,
     80,
     86,
     93,
     94,
     95,
     100,
     104,
     106,
     109,
     114,
     116,
     118,
     120,
     126,
     129,
     131,
     133,
     135,
     137,
     140,
     142,
     145,
     147,
     150,
     152,
     154,
     156,
     159,
     161,
     163,
     165,
     167,
     169,
     171,
     173,
     175,
     178,
     180,
     191,
     193,
     195,
     199,
     201,
     203,
     215,
     216,
     217,
     218,
     219,
     222,
     227,
     228,
     230,
     232,
     233,
     234,
     235,
     237,
     241,
     242,
     243,
     244,
     245,
     257,
     260,
     262,
     269,
     280,
     288,
     294,
     297,
     300,
     304,
     306,
     310,
     311,
     312,
     318,
     319,
     323,
     332,
     345],
 1: [24,
     32,
     35,
     41,
     

In [53]:
centers = km_model.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

AttributeError: 'KMeans' object has no attribute 'cluster_centers_'

In [54]:
pprint(km_model)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)


In [55]:
print(clusters)

defaultdict(<class 'list'>, {6: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 209, 210, 211, 212, 213, 214, 352, 353, 354, 355, 356], 0: [1, 27, 30, 33, 38, 43, 46, 47, 49, 51, 54, 56, 60, 66, 71, 79, 80, 86, 93, 94, 95, 100, 104, 106, 109, 114, 116, 118, 120, 126, 129, 131, 133, 135, 137, 140, 142, 145, 147, 150, 152, 154, 156, 159, 161, 163, 165, 167, 169, 171, 173, 175, 178, 180, 191, 193, 195, 199, 201, 203, 215, 216, 217, 218, 219, 222, 227, 228, 230, 232, 233, 234, 235, 237, 241, 242, 243, 244, 245, 257, 260, 262, 269, 280, 288, 294, 297, 300, 304, 306, 310, 311, 312, 318, 319, 323, 332, 345], 5: [16, 17, 19, 20, 21, 22, 23, 25, 28, 29, 34, 36, 37, 40, 42, 57, 58, 62, 63, 64, 67, 72, 77, 81, 82, 83, 85, 87, 92, 96, 97, 107, 117, 124, 130, 134, 138, 141, 143, 148, 149, 151, 153, 155, 157, 160, 164, 166, 170, 176, 177, 181, 184, 190, 194, 196, 221, 224, 226, 229, 231, 240, 246, 250, 251, 252, 254, 256, 258, 263, 264, 265, 267, 268, 270, 271, 272, 273, 276, 282, 283, 284, 286,

In [56]:
for x in clusters:
    print (x)
    for y in clusters[x]:
        print (y,':',clusters[x][y])

6
0 : 0
2 : 3
3 : 4
4 : 5
5 : 6
6 : 7
7 : 8
8 : 9
9 : 10
10 : 11
11 : 12
12 : 13
13 : 14
14 : 15
15 : 209


IndexError: list index out of range

In [57]:
import json
print(json.dumps(clusters, indent = 4))

TypeError: keys must be str, int, float, bool or None, not int32